In [1]:
import tensorflow as tf
print("Tensorflow version", tf.__version__)

Tensorflow version 1.6.0


In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [9]:
def cnn_lstm_model(features, labels, mode):
    # Input layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu
    )
    
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1,
        pool_size=[2, 2],
        strides=2
    )
    
    # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu
    )
    
    # Pooling Layer #2
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2,
        pool_size=[2, 2],
        strides=2
    )
    
    # Convolutionnal Layer #3 to shrink Y axis to size 1
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=128,
        kernel_size=[1, 7],
        padding="valid",
        activation=tf.nn.relu
    )
    
    # Squeeze Y axis
    conv3_squeezed = tf.squeeze(conv3)

    # LSTM Layer
    lstm_input = conv3_squeezed
    lstm_cell = tf.nn.rnn_cell.LSTMCell(256, state_is_tuple=True)
    lstm_outputs, lstm_state = tf.nn.dynamic_rnn(lstm_cell, lstm_input, dtype=tf.float32)
    lstm1 = lstm_state.h
    
    # Fully connected
    dense = tf.layers.dense(inputs=lstm1, units=500, activation=tf.nn.relu)
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # Logits
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    # Predictions
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = { "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]) }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [10]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [11]:
# Creating Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_lstm_model, model_dir="/tmp/mnist_cnn_lstm_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_cnn_lstm_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9fc5a3fba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
# Logging
logging_hook = tf.train.LoggingTensorHook(tensors={"probabilities": "softmax_tensor"}, every_n_iter=1000)

In [13]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=5000,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_cnn_lstm_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.10114493 0.1005638  0.0995631  0.10112219 0.10082104 0.09998313
  0.09943858 0.09775923 0.10079513 0.09880891]
 [0.09988782 0.09961054 0.09972957 0.10060175 0.10106754 0.09912646
  0.10153967 0.09696423 0.101016   0.10045645]
 [0.0996571  0.09940007 0.10025741 0.10014533 0.10145752 0.1004082
  0.10032212 0.09824052 0.09956422 0.10054747]
 [0.09960882 0.10096332 0.0994474  0.10073671 0.1015017  0.09966242
  0.10025642 0.09711184 0.10087264 0.09983873]
 [0.0995009  0.10001466 0.09952343 0.10042772 0.10136357 0.09946063
  0.10114735 0.09841826 0.10080307 0.09934041]
 [0.09991834 0.10074084 0.10006473 0.10024989 0.10130183 0.10001855
  0.1

INFO:tensorflow:loss = 2.3022711, step = 1
INFO:tensorflow:global_step/sec: 43.6405
INFO:tensorflow:loss = 2.2897005, step = 101 (2.278 sec)
INFO:tensorflow:global_step/sec: 54.7307
INFO:tensorflow:loss = 2.2059577, step = 201 (1.830 sec)
INFO:tensorflow:global_step/sec: 52.6893
INFO:tensorflow:loss = 1.6699423, step = 301 (1.898 sec)
INFO:tensorflow:global_step/sec: 51.4662
INFO:tensorflow:loss = 1.4955771, step = 401 (1.944 sec)
INFO:tensorflow:global_step/sec: 48.6393
INFO:tensorflow:loss = 1.0159934, step = 501 (2.058 sec)
INFO:tensorflow:global_step/sec: 48.5495
INFO:tensorflow:loss = 0.48560736, step = 601 (2.059 sec)
INFO:tensorflow:global_step/sec: 49.4159
INFO:tensorflow:loss = 0.6376366, step = 701 (2.023 sec)
INFO:tensorflow:global_step/sec: 49.011
INFO:tensorflow:loss = 0.6240351, step = 801 (2.040 sec)
INFO:tensorflow:global_step/sec: 47.2331
INFO:tensorflow:loss = 0.39580497, step = 901 (2.117 sec)
INFO:tensorflow:global_step/sec: 48.7497
INFO:tensorflow:probabilities = [

INFO:tensorflow:loss = 0.5110937, step = 1001 (2.089 sec)
INFO:tensorflow:global_step/sec: 43.7091
INFO:tensorflow:loss = 0.19732237, step = 1101 (2.250 sec)
INFO:tensorflow:global_step/sec: 51.771
INFO:tensorflow:loss = 0.2321011, step = 1201 (1.932 sec)
INFO:tensorflow:global_step/sec: 51.9411
INFO:tensorflow:loss = 0.19900058, step = 1301 (1.925 sec)
INFO:tensorflow:global_step/sec: 51.6043
INFO:tensorflow:loss = 0.1575113, step = 1401 (1.938 sec)
INFO:tensorflow:global_step/sec: 48.2421
INFO:tensorflow:loss = 0.13698123, step = 1501 (2.073 sec)
INFO:tensorflow:global_step/sec: 48.7263
INFO:tensorflow:loss = 0.11114948, step = 1601 (2.052 sec)
INFO:tensorflow:global_step/sec: 51.2033
INFO:tensorflow:loss = 0.07628069, step = 1701 (1.953 sec)
INFO:tensorflow:global_step/sec: 52.4163
INFO:tensorflow:loss = 0.085577235, step = 1801 (1.908 sec)
INFO:tensorflow:global_step/sec: 51.75
INFO:tensorflow:loss = 0.07801047, step = 1901 (1.932 sec)
INFO:tensorflow:global_step/sec: 50.1163
INFO:

INFO:tensorflow:loss = 0.057470158, step = 2001 (2.030 sec)
INFO:tensorflow:global_step/sec: 51.0469
INFO:tensorflow:loss = 0.18756782, step = 2101 (1.925 sec)
INFO:tensorflow:global_step/sec: 44.3343
INFO:tensorflow:loss = 0.07749465, step = 2201 (2.259 sec)
INFO:tensorflow:global_step/sec: 49.2186
INFO:tensorflow:loss = 0.13168396, step = 2301 (2.028 sec)
INFO:tensorflow:global_step/sec: 51.5109
INFO:tensorflow:loss = 0.10525056, step = 2401 (1.941 sec)
INFO:tensorflow:global_step/sec: 51.9617
INFO:tensorflow:loss = 0.07054437, step = 2501 (1.925 sec)
INFO:tensorflow:global_step/sec: 51.7791
INFO:tensorflow:loss = 0.11995098, step = 2601 (1.931 sec)
INFO:tensorflow:global_step/sec: 51.4786
INFO:tensorflow:loss = 0.16254032, step = 2701 (1.943 sec)
INFO:tensorflow:global_step/sec: 51.3311
INFO:tensorflow:loss = 0.057366956, step = 2801 (1.948 sec)
INFO:tensorflow:global_step/sec: 52.5277
INFO:tensorflow:loss = 0.0585898, step = 2901 (1.904 sec)
INFO:tensorflow:global_step/sec: 48.6948

INFO:tensorflow:loss = 0.06657903, step = 3001 (2.085 sec)
INFO:tensorflow:global_step/sec: 50.6488
INFO:tensorflow:loss = 0.044330765, step = 3101 (1.942 sec)
INFO:tensorflow:global_step/sec: 50.9648
INFO:tensorflow:loss = 0.1092982, step = 3201 (1.962 sec)
INFO:tensorflow:global_step/sec: 51.7722
INFO:tensorflow:loss = 0.018790139, step = 3301 (1.931 sec)
INFO:tensorflow:global_step/sec: 51.6224
INFO:tensorflow:loss = 0.097459905, step = 3401 (1.937 sec)
INFO:tensorflow:global_step/sec: 52.2669
INFO:tensorflow:loss = 0.055563066, step = 3501 (1.913 sec)
INFO:tensorflow:global_step/sec: 51.8261
INFO:tensorflow:loss = 0.021567764, step = 3601 (1.930 sec)
INFO:tensorflow:global_step/sec: 50.9998
INFO:tensorflow:loss = 0.047958627, step = 3701 (1.960 sec)
INFO:tensorflow:global_step/sec: 51.7947
INFO:tensorflow:loss = 0.071318336, step = 3801 (1.931 sec)
INFO:tensorflow:global_step/sec: 52.2441
INFO:tensorflow:loss = 0.07199035, step = 3901 (1.915 sec)
INFO:tensorflow:global_step/sec: 50

INFO:tensorflow:loss = 0.021371556, step = 4001 (2.011 sec)
INFO:tensorflow:global_step/sec: 51.4859
INFO:tensorflow:loss = 0.091250114, step = 4101 (1.913 sec)
INFO:tensorflow:global_step/sec: 52.5615
INFO:tensorflow:loss = 0.09667393, step = 4201 (1.902 sec)
INFO:tensorflow:global_step/sec: 51.3229
INFO:tensorflow:loss = 0.073478304, step = 4301 (1.948 sec)
INFO:tensorflow:global_step/sec: 51.6803
INFO:tensorflow:loss = 0.04838639, step = 4401 (1.936 sec)
INFO:tensorflow:global_step/sec: 51.4281
INFO:tensorflow:loss = 0.057732113, step = 4501 (1.943 sec)
INFO:tensorflow:global_step/sec: 51.5984
INFO:tensorflow:loss = 0.122740984, step = 4601 (1.939 sec)
INFO:tensorflow:global_step/sec: 51.6091
INFO:tensorflow:loss = 0.027637722, step = 4701 (1.938 sec)
INFO:tensorflow:global_step/sec: 51.2503
INFO:tensorflow:loss = 0.010799813, step = 4801 (1.952 sec)
INFO:tensorflow:global_step/sec: 51.0856
INFO:tensorflow:loss = 0.0110158725, step = 4901 (1.957 sec)
INFO:tensorflow:Saving checkpoin

In [14]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.


ValueError: as_list() is not defined on an unknown TensorShape.